In [30]:
import torch
from torch import nn
from tqdm.auto import tqdm
from torchvision import transforms
import matplotlib.pyplot as plt
from torchtext.vocab import vocab
from torchtext.data import get_tokenizer
from collections import OrderedDict

torch.manual_seed(0) # Set for testing purposes, please do not change!

In [31]:
def get_vocab(corpora):
    tokens = set()
    tokenizer = get_tokenizer(None)
    for line in corpora:
        words = tokenizer(line)
        for word in words:
            for token in word:
                tokens.add(token)
    v1 = vocab(OrderedDict([(token, 1) for token in tokens]), specials=["<end>", "<pad>"])
    return v1

In [32]:
corpora = """Perda de pontos no futebol por racismo de torcedores ainda é realidade distante
Brasil é o único país entre as principais ligas a prever a punição esportiva, que ainda não foi efetivamente aplicada
Compartilhar nas redes sociais "Tostão: Existe uma inversão no aprendizado esportivo"
Crianças precisam se divertir com a bola antes de aprender técnica e regras
Compartilhar nas redes sociais "Financial Times: Netflix fecha acordo de R$ 24,7 bilhões para transmitir luta livre nos EUA"
Netflix fecha acordo de R$ 24,7 bilhões para transmitir luta livre nos EUA
Serviço de streaming exibirá a atração a partir de 2025 como parte de um acordo de dez anos
Compartilhar nas redes sociais "Futebol Internacional: Udinese fará jogo com portões fechados como punição por insultos a Maignan"
Udinese fará jogo com portões fechados como punição por insultos a Maignan
Goleiro francês Mike Maignan, do Milan, foi alvo de insultos por parte de torcedores do time de Udine
Compartilhar nas redes sociais "Esporte: Corinthians e Cruzeiro disputam título da Copinha na Neo Química Arena"
Corinthians e Cruzeiro disputam título da Copinha na Neo Química Arena
Partida decisiva acontece nesta quinta-feira (25), aniversário de São Paulo
Compartilhar nas redes sociais "Como É Que É?: Bets: por que apostas online viraram um fenômeno no Brasil?"
Bets: por que apostas online viraram um fenômeno no Brasil?
Isabella Faria conversa com João Gabriel às 18h, ao vivo
Compartilhar nas redes sociais "O Mundo É uma Bola: Os campeões da preferência e da resistência"
Bruno Fernandes, do Manchester United, esteve em campo 76 vezes em 2023, cinco a mais que Gustavo Gómez, do Palmeiras, e Rodrygo, do Real Madrid
Compartilhar nas redes sociais "Esporte: Gigi Riva, lenda do futebol italiano, morre aos 79 anos"
Gigi Riva, lenda do futebol italiano, morre aos 79 anos
Jogador foi campeão europeu em 1968 e vice-campeão mundial em 1970
Compartilhar nas redes sociais "Sandro Macedo: O efervescente mercadão de janeiro no futebol"
Janela de transferências dá a impressão de que clubes brasilianos são mais opulentos do que realmente são
Compartilhar nas redes sociais "Esporte: Equipe brasileira de curling vence pela primeira vez nas Olimpíadas de Inverno"
Equipe brasileira de curling vence pela primeira vez nas Olimpíadas de Inverno
Modalidade é conhecida pelo uso de uma espécie de vassoura para deixar o gelo mais liso
Compartilhar nas redes sociais "Futebol Internacional: Presidente da Fifa defende derrota automática em casos de racismo nos estádios"
Presidente da Fifa defende derrota automática em casos de racismo nos estádios
Manifestações contra jogadores negros ocorreram na Itália e na Inglaterra no sábado
Compartilhar nas redes sociais "Café da Manhã: Podcast discute a regulamentação e o impacto das apostas esportivas online"
Podcast discute a regulamentação e o impacto das apostas esportivas online
Bets chegam a 15% da população, segundo o Datafolha; governo mira arrecadação, e especialistas alertam para riscos
Compartilhar nas redes sociais "Campeonato Paulista: Romero marca de voleio, e Corinthians estreia com vitória no Paulista"
Romero marca de voleio, e Corinthians estreia com vitória no Paulista"""

In [33]:
v1 = get_vocab(corpora)

In [399]:
class Autoencoder(nn.Module):
    def __init__(self, char_num_embeddings=10, char_embedding_dim=5, word_embedding_dim=2, word_chars_size=5):
        super(Autoencoder, self).__init__()
        self.char_num_embeddings = char_num_embeddings
        self.char_embedding_dim = char_embedding_dim
        self.word_embedding_dim = word_embedding_dim
        self.word_chars_size = word_chars_size
        self.embedding = nn.Sequential(
            nn.Embedding(char_num_embeddings, char_embedding_dim, max_norm=True)
        )
        self.encoder = nn.Sequential(
            nn.Conv2d(1, 8, kernel_size=(3, char_embedding_dim), padding=(2, 0)),
            nn.ReLU(True),
            nn.Conv2d(8, 16, kernel_size=(3, 1), padding=(2, 0)),
            nn.Flatten(1, 3),
            nn.Linear((self.word_chars_size + 4) * 16, word_embedding_dim)
        )
        self.decoder = nn.Sequential(
            nn.Linear(word_embedding_dim, (self.word_chars_size + 4) * 16),
            nn.Unflatten(1, (16, (self.word_chars_size + 4), 1)),
            nn.ConvTranspose2d(16, (self.word_chars_size + 4), kernel_size=(1, 1)),
            nn.ReLU(True),
            nn.ConvTranspose2d((self.word_chars_size + 4), 1, kernel_size=(1, char_embedding_dim), padding=(2, 0)),
        )
    def forward(self, x):
        x = x.view(len(x), 1, self.word_chars_size)
        return self.decoder(self.encoder(self.embedding(x)))

In [429]:
class CBOW(nn.Module):
    def __init__(self, autoencoder, context_size=2, hidden_dim=16):
        super(CBOW, self).__init__()
        self.context_size = context_size
        self.autoencoder = autoencoder
        self.cbow = nn.Sequential(
            nn.Linear(context_size * self.autoencoder.word_embedding_dim, hidden_dim),
            nn.BatchNorm1d(hidden_dim),
            nn.ReLU(inplace=True),
            nn.Linear(hidden_dim, self.autoencoder.char_num_embeddings * self.autoencoder.word_chars_size),
        )
    def forward(self, x):
        size = len(x)
        x = x.view(size, self.context_size, self.autoencoder.word_chars_size)
        x = self.autoencoder.embedding(x)
        x = x.view(size * self.context_size, 1, self.autoencoder.word_chars_size, self.autoencoder.char_embedding_dim)
        x = self.autoencoder.encoder(x)
        x = x.view(size, self.context_size * self.autoencoder.word_embedding_dim)
        x = self.cbow(x)
        x = x.view(size, self.autoencoder.word_chars_size, self.autoencoder.char_num_embeddings)
        x = F.log_softmax(x, dim=2)
        return x

In [430]:
device = 'cpu'
lr = 0.00001
char_num_embeddings = len(v1)  # size of the dictionary of embeddings
char_embedding_dim = 8  # dimension of the char embedding
word_embedding_dim = 2  # dimension of the word embedding
word_chars_size = 6  # size of chars in a word
context_size = 3  # size of words to use as context in CBOW

gen = Autoencoder(
    char_num_embeddings=char_num_embeddings,
    char_embedding_dim=char_embedding_dim,
    word_embedding_dim=word_embedding_dim,
    word_chars_size=word_chars_size,
    ).to(device)
gen_opt = torch.optim.Adam(gen.parameters(), lr=lr)

cbow = CBOW(gen, context_size=context_size)

In [432]:
cbow(torch.LongTensor([[[1, 2, 4, 5, 9, 1], [1, 2, 4, 5, 10, 1], [1, 2, 4, 5, 10, 1]], [[1, 2, 4, 5, 11, 1], [1, 2, 4, 5, 10, 1], [1, 2, 3, 5, 12, 1]]]))

tensor([[[-4.2246, -4.3383, -4.1644, -4.2291, -4.7547, -4.4346, -3.4667,
          -4.5913, -4.4640, -4.4851, -4.1680, -3.7681, -4.5705, -4.7998,
          -4.5585, -3.9025, -4.7107, -4.0311, -4.3334, -4.1878, -4.3373,
          -4.2947, -4.3054, -5.0092, -4.1335, -4.3985, -4.1418, -4.3437,
          -4.8272, -4.6650, -4.3460, -4.4781, -4.3251, -4.3156, -4.4948,
          -4.1904, -3.9926, -4.6989, -4.1742, -4.5645, -4.3902, -4.2616,
          -4.0615, -4.2615, -4.7062, -4.3915, -4.2499, -4.3453, -4.5480,
          -4.2700, -4.2231, -4.7090, -4.9333, -4.4629, -4.4482, -4.7066,
          -4.2133, -4.4751, -4.5909, -4.6999, -4.4857, -4.5342, -4.3695,
          -3.7433, -4.1867, -4.3822, -4.5237, -4.8014, -4.1360, -4.3467,
          -4.7781, -4.9099, -4.5954, -4.6759, -4.0770, -4.1481, -3.8628],
         [-4.5196, -4.4608, -4.4592, -4.3055, -3.8292, -4.1279, -4.4441,
          -4.4805, -4.2939, -3.9630, -4.3435, -4.1682, -4.3537, -4.3206,
          -4.5413, -3.8988, -4.1380, -4.3173, -4.4

In [259]:
import torch.nn.functional as F
import torch.optim as optim

CONTEXT_SIZE = 2
EMBEDDING_DIM = 10
# We will use Shakespeare Sonnet 2
test_sentence = """When forty winters shall besiege thy brow,
And dig deep trenches in thy beauty's field,
Thy youth's proud livery so gazed on now,
Will be a totter'd weed of small worth held:
Then being asked, where all thy beauty lies,
Where all the treasure of thy lusty days;
To say, within thine own deep sunken eyes,
Were an all-eating shame, and thriftless praise.
How much more praise deserv'd thy beauty's use,
If thou couldst answer 'This fair child of mine
Shall sum my count, and make my old excuse,'
Proving his beauty by succession thine!
This were to be new made when thou art old,
And see thy blood warm when thou feel'st it cold.""".split()
# we should tokenize the input, but we will ignore that for now
# build a list of tuples.
# Each tuple is ([ word_i-CONTEXT_SIZE, ..., word_i-1 ], target word)
ngrams = [
    (
        [test_sentence[i - j - 1] for j in range(CONTEXT_SIZE)],
        test_sentence[i]
    )
    for i in range(CONTEXT_SIZE, len(test_sentence))
]
# Print the first 3, just so you can see what they look like.
print(ngrams[:3])

vocab = set(test_sentence)
word_to_ix = {word: i for i, word in enumerate(vocab)}


class NGramLanguageModeler(nn.Module):

    def __init__(self, vocab_size, embedding_dim, context_size):
        super(NGramLanguageModeler, self).__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.linear1 = nn.Linear(context_size * embedding_dim, 128)
        self.linear2 = nn.Linear(128, vocab_size)

    def forward(self, inputs):
        embeds = self.embeddings(inputs).view((1, -1))
        out = F.relu(self.linear1(embeds))
        out = self.linear2(out)
        log_probs = F.log_softmax(out, dim=1)
        return log_probs


losses = []
loss_function = nn.NLLLoss()
model = NGramLanguageModeler(len(vocab), EMBEDDING_DIM, CONTEXT_SIZE)
optimizer = optim.SGD(model.parameters(), lr=0.001)

for epoch in range(10):
    total_loss = 0
    for context, target in ngrams:

        # Step 1. Prepare the inputs to be passed to the model (i.e, turn the words
        # into integer indices and wrap them in tensors)
        context_idxs = torch.tensor([word_to_ix[w] for w in context], dtype=torch.long)
        raise Exception(context_idxs)

        # Step 2. Recall that torch *accumulates* gradients. Before passing in a
        # new instance, you need to zero out the gradients from the old
        # instance
        model.zero_grad()

        # Step 3. Run the forward pass, getting log probabilities over next
        # words
        log_probs = model(context_idxs)

        # Step 4. Compute your loss function. (Again, Torch wants the target
        # word wrapped in a tensor)
        loss = loss_function(log_probs, torch.tensor([word_to_ix[target]], dtype=torch.long))

        # Step 5. Do the backward pass and update the gradient
        loss.backward()
        optimizer.step()

        # Get the Python number from a 1-element Tensor by calling tensor.item()
        total_loss += loss.item()
    losses.append(total_loss)
print(losses)  # The loss decreased every iteration over the training data!

# To get the embedding of a particular word, e.g. "beauty"
print(model.embeddings.weight[word_to_ix["beauty"]])

[(['forty', 'When'], 'winters'), (['winters', 'forty'], 'shall'), (['shall', 'winters'], 'besiege')]


Exception: tensor([18, 28])

In [29]:
model(torch.tensor([18, 28, 45]))

RuntimeError: mat1 and mat2 shapes cannot be multiplied (1x30 and 20x128)

In [217]:
torch.tensor([word_to_ix[target]])

tensor([96])